In [1]:
import os

import numpy as np
import math
import random

import torch

import torch.nn as nn 
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

from brevitas.export import export_onnx_qcdq

import config_aimet
import models_aimet_medium_fasdd
import models
import utils
import datasets
import metrics
import loss
import val_epoch

In [2]:
if config_aimet.MODEL == "BED":
    
    print("Using Fixed Point Quantizers without BN")
    quant_model = models_aimet_medium_fasdd.QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER(
            weight_bw = config_aimet.WEIGHTS_BIT_WIDTH,
            big_layers_weight_bw = config_aimet.BIG_LAYERS_WEIGHTS_BIT_WIDTH,
            act_bw = config_aimet.ACTIVATIONS_BIT_WIDTH,
            bias_bw = config_aimet.BIAS_BIT_WIDTH,
            num_classes=config_aimet.N_CLASSES).to(config_aimet.DEVICE)
    # quant_model = models.QUANT_FixedPoint_NoBN_BED_CLASSIFIER(
    #         weight_bw = config_aimet.WEIGHTS_BIT_WIDTH,
    #         big_layers_weight_bw = config_aimet.BIG_LAYERS_WEIGHTS_BIT_WIDTH,
    #         act_bw = config_aimet.ACTIVATIONS_BIT_WIDTH,
    #         bias_bw = config_aimet.BIAS_BIT_WIDTH,
    #         num_classes=config_aimet.N_CLASSES).to(config_aimet.DEVICE)

else:
    print("Wrong Model")
    raise SystemExit("Wrong Model")

optimizer = optim.Adam(quant_model.parameters(), 
                       lr=config_aimet.LEARNING_RATE, 
                       weight_decay=config_aimet.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config_aimet.FACTOR, 
                                                 patience=config_aimet.PATIENCE, 
                                                 threshold=config_aimet.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config_aimet.MIN_LR)

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in quant_model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(quant_model.parameters()).numel()
print(f'Total parameters = {n_params}\n')

Using Fixed Point Quantizers without BN

Trainable parameters = 63631
Total parameters = 63631



## Model Medium Compression

In [3]:
# model_dir = 'experiments_fuseBN_256_fasdd/test_v10_MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
# model_file = model_dir + 'BED_classifier__best_mean_F1.pt'
# epochs_trained = utils.load_checkpoint(model_file, quant_model, optimizer, scheduler, config_aimet.DEVICE)

### Model with Medium Compression: conv341 defined as big layer

In [4]:
# model_dir = 'experiments_fuseBN_256_fasdd/test_v11_MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
# model_file = model_dir + 'BED_classifier__smoke__precision=0.9096__recall=0.8942__epoch=93.pt'
# epochs_trained = utils.load_checkpoint(model_file, quant_model, optimizer, scheduler, config_aimet.DEVICE)

### FASDD VAL added: Model with Medium Compression: conv341 defined as big layer

In [5]:
model_dir = 'experiments_fuseBN_256_fasdd/test_v30_FASDD_VAL__MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
model_file = model_dir + 'BED_classifier__best_mean_F1.pt'
epochs_trained = utils.load_checkpoint(model_file, quant_model, optimizer, scheduler, config_aimet.DEVICE)

Loading Model. Trained during 94 epochs


## Model with No Compression

In [6]:
# model_dir = 'experiments_fuseBN_256_fasdd/test_v00_NoCOMP_w4W2a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
# model_file = model_dir + 'BED_classifier__best_smoke__precision=0.9217__epoch=59.pt'
# epochs_trained = utils.load_checkpoint(model_file, quant_model, optimizer, scheduler, config_aimet.DEVICE)

# Evaluate Brevitas Quant Model

## Dataset

In [7]:
val_loader = datasets.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 1997
FASDD only smoke images: 846
FASDD only fire images: 35
FASDD smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 6533
FASDD only smoke images: 3902
FASDD only fire images: 2091
FASDD smoke and fire images: 3358

Test FASDD CV dataset len: 15884
Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 8487
Concatenate with FASDD CV dataset
Test dataset len: 24371


## Loss
Needed for evaluation function

In [8]:
if config_aimet.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    print(f'Smoke Precision Weight: {config_aimet.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config_aimet.DEVICE, smoke_precision_weight=config_aimet.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


In [9]:
quant_model.eval()
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=quant_model,                         
        loss_fn=loss_fn,
        device=config_aimet.DEVICE)

Validating:   0%|                                                                                                                     | 0/380 [00:00<?, ?it/s]/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)
Validating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:54<00:00,  6.93it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
16.312      |10.550      |5.762       
SMOKE -> Precision: 0.950 - Recall: 0.897 - Accuracy: 0.929 - F1: 0.923
FIRE -> Precision: 0.926 - Recall: 0.974 - Accuracy: 0.966 - F1: 0.950


In [10]:
for k, v in val_metrics.items():
    print(f'{k}: smoke {v[0]:.4f} - fire: {v[1]:.4f}')
print(f'F1 Mean: {(val_metrics["F1"][0] + val_metrics["F1"][1])/2:.4f}')

Accuracy: smoke 0.9292 - fire: 0.9665
Precision: smoke 0.9499 - fire: 0.9262
Recall: smoke 0.8971 - fire: 0.9743
F1: smoke 0.9227 - fire: 0.9496
F1 Mean: 0.9362


# Model to CPU and ONNX Export

In [11]:
quant_model.to('cpu')
print("Model to CPU before exports")

Model to CPU before exports


In [11]:
# export_onnx_qcdq(
#     quant_model, 
#     torch.randn(1, 3, config_aimet.IMG_H, config_aimet.IMG_W).to('cpu'), 
#     export_path='./models/onnx_export/medium_fassd__conv341_big__epoch=93.onnx')

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/brevitas/export/onnx/standard/manager.py:26: UserWarning: ONNX opset version set to 13, override with opset_version=
  warnings.warn(f"ONNX opset version set to {DEFAULT_OPSET}, override with {ka}=")


# Export to QONNX, previous step to FINN-ONNX

In [12]:
from brevitas.export import export_qonnx

In [14]:
export_qonnx(
    quant_model, 
    export_path='./experiments_fuseBN_256_fasdd/test_v30_FASDD_VAL__MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/BED__med_comp__341_big__best_mean_f1__cpu__QONNX.onnx',
    input_t=torch.randn(1, 3, config_aimet.IMG_H, config_aimet.IMG_W).to('cpu') 
)

ir_version: 8
producer_name: "pytorch"
producer_version: "2.1.2"
graph {
  node {
    input: "x.1823"
    input: "/model/input0/act_quant/export_handler/Constant_1_output_0"
    input: "/model/input0/act_quant/export_handler/Constant_2_output_0"
    input: "/model/input0/act_quant/export_handler/Constant_output_0"
    output: "/model/input0/act_quant/export_handler/Quant_output_0"
    name: "/model/input0/act_quant/export_handler/Quant"
    op_type: "Quant"
    attribute {
      name: "narrow"
      i: 0
      type: INT
    }
    attribute {
      name: "rounding_mode"
      s: "ROUND"
      type: STRING
    }
    attribute {
      name: "signed"
      i: 0
      type: INT
    }
    domain: "onnx.brevitas"
  }
  node {
    input: "/model/conv1/weight_quant/export_handler/Constant_1_output_0"
    input: "/model/conv1/weight_quant/export_handler/Constant_2_output_0"
    input: "/model/input0/act_quant/export_handler/Constant_2_output_0"
    input: "/model/conv1/weight_quant/export_handle